In [10]:
import pandas as pd
import psycopg2

from tqdm import tqdm

In [18]:
# Read local data and concatenate it into one dataset
df_interfax = pd.read_csv('interfax_parsed_data.csv', index_col=0)

df_smart_labs = pd.read_csv('smart_labs_parced_data.csv')
df_smart_labs.rename(columns={'key_words': 'tags'}, inplace=True)

df_ria = pd.read_csv('ria_parsed_data.csv', index_col=0)

df_kommersant = pd.read_csv('kommersant_parced_data.csv')
df_kommersant.rename(columns={'key_words': 'tags'}, inplace=True)

df = pd.concat(
    [df_interfax, df_smart_labs, df_ria, df_kommersant],
    axis=0, 
    ignore_index=True
)

In [12]:
# Connection details
dbname = 'defaultdb'
user = 'doadmin'
password = 'AVNS_pWw7oK1YPADBn-TRPqp'
host = 'db-postgresql-fra1-12797-do-user-8775628-0.c.db.ondigitalocean.com'
port = '25060'

conn = psycopg2.connect(f"dbname={dbname} user={user} password={password} host={host} port={port}")
cursor = conn.cursor()

In [13]:
try:
    # Create new table for the data
    query = '''CREATE TABLE raw_data (
        id SERIAL PRIMARY KEY,
        website TEXT,
        section TEXT,
        url TEXT,
        header TEXT,
        body TEXT,
        date TEXT,
        tags TEXT);'''
    cursor.execute(query)
    
    # Check of created table's schema
    query = '''SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'raw_data';'''
    cursor.execute(query)
    print(cursor.fetchall())
    
    # Push data to created table
    for index, row in tqdm(df.iterrows()):
        if type(row.tags) is not float:
            row.tags = row.tags.replace('\'','"')
        if type(row.header) is not float:
            row.header = row.header.replace('\'','"')
        if type(row.body) is not float:
            row.body = row.body.replace('\'','"')
        cursor.execute(
             "INSERT INTO public.raw_data (website, section, url, header, body, date, tags)\
             VALUES ('{}','{}','{}','{}','{}','{}', '{}');".\
            format(row.website, row.section, row.url, row.header, row.body, row.date, row.tags)
         )
    
    # Commit changes
    conn.commit()
    
except psycopg2.Error as e:
    print(f"Error: {e}")
    conn.rollback() 
    
finally:
    if conn:
        cursor.close()
        conn.close()

[('id', 'integer'), ('website', 'text'), ('section', 'text'), ('url', 'text'), ('header', 'text'), ('body', 'text'), ('date', 'text'), ('tags', 'text')]


45721it [39:34, 19.25it/s]


In [17]:
%%time
# Naive check that everything is fine
conn = psycopg2.connect(f"dbname={dbname} user={user} password={password} host={host} port={port}")
df = pd.read_sql_query('SELECT * FROM raw_data', conn)
conn.close()

df

<timed exec>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


CPU times: total: 3 s
Wall time: 25.4 s


,id,website,section,url,header,body,date,tags
0,1,Интерфакс,В РОССИИ,https://www.interfax.ru/russia/879448,Банки РФ с 2023 года начнут выходить из регуля...,Москва. 1 января. INTERFAX.RU - Банк России с ...,"07:03, 1 января 2023","[""Россия"", ""банки"", ""послабления"", ""отмена""]"
1,2,Интерфакс,В РОССИИ,https://www.interfax.ru/russia/879447,Запрет на раскрытие информации для банков в РФ...,Москва. 1 января. INTERFAX.RU - Банк России пр...,"06:23, 1 января 2023","[""Россия"", ""банки"", ""отчетность"", ""раскрытие""]"
2,3,Интерфакс,В РОССИИ,https://www.interfax.ru/russia/879440,Расширен перечень налоговых льгот для резидент...,Москва. 1 января. INTERFAX.RU - Резиденты росс...,"05:08, 1 января 2023","[""Россия"", ""налоги"", ""ставки"", ""САР""]"
3,4,Интерфакс,В РОССИИ,https://www.interfax.ru/russia/879436,С 1 января будет взиматься пошлина при экспорт...,Москва. 1 января. INTERFAX.RU - Пошлина при эк...,"04:10, 1 января 2023","[""Россия"", ""топливо"", ""демпфер"", ""пошлина""]"
4,5,Интерфакс,В РОССИИ,https://www.interfax.ru/russia/879434,Пошлина на экспорт подсолнечного масла из РФ в...,На шрот - вырастет почти в 23 раза\nМосква. 1 ...,"04:05, 1 января 2023","[""Россия"", ""экспорт"", ""масло"", ""пошлина""]"
...,...,...,...,...,...,...,...,...
45716,45713,Kommersant,Финансы,https://www.kommersant.ru/doc/5623234,Долговая нагрузка россиян во втором квартале 2...,Долговая нагрузка российских граждан во II ква...,"21.10.2022, 12:34",[]
45717,45714,Kommersant,Финансы,https://www.kommersant.ru/doc/5622262,РБК: турецкие банки усложнили процедуру выдачи...,Турецкие банки не справляются с наплывом гражд...,"21.10.2022, 10:27",[]
45718,45715,Kommersant,Финансы,https://www.kommersant.ru/doc/5622210,Индонезия обсуждает с Россией прием карт «Мир»,Москва и Джакарта обсуждают возможность приема...,"21.10.2022, 08:48",[]
45719,45716,Kommersant,Финансы,https://www.kommersant.ru/doc/5622206,Счетная палата: средняя пенсия к 2026 году буд...,В ближайшие годы пенсии в России будут расти с...,"21.10.2022, 08:37",[]
